In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
sys.path.insert(0, "../../code-previous")

In [4]:
import pandas as pd
import numpy as np
import operator

import peakutils

import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')

import operator


import pickle
from sklearn.feature_extraction.text import CountVectorizer

import plotly
import plotly.plotly as py
from plotly import tools

import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

In [5]:
filename = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/word-journey-data.pql"
#data_grouped_by_year_label.to_pickle(filename)


data_grouped_by_year_label = pd.read_pickle(filename)

In [6]:
data_grouped_by_year_label.head()

,index,Label,PubYear,title_without_stopwords,cleaned_titles,title_words_,title_words
0,0,Acoustics,1929,"[hearing speech auditoriums, hearing speech au...",[psychological measurements annoyance related ...,psychological measurements annoyance related p...,psychological measurements annoyance related p...
1,1,Acoustics,1930,[reverberation time bridge instrumental metho...,"[theory horn type loud speaker, discussion sym...",theory horn type loud speaker discussion sympo...,theory horn type loud speaker discussion sympo...
2,2,Acoustics,1931,"[some physical characteristics speech music, s...","[microphone technique radio broadcasting, orga...",microphone technique radio broadcasting organ ...,microphone technique radio broadcasting organ ...
3,3,Acoustics,1932,"[position finding under water sound signals, p...",[acoustic pick up philadelphia orchestra broad...,acoustic pick up philadelphia orchestra broadc...,acoustic pick up philadelphia orchestra broadc...
4,4,Acoustics,1933,"[some notes character bell tones, some notes c...","[sound absorption non reactive mixtures, deter...",sound absorption non reactive mixtures determi...,sound absorption non reactive mixtures determi...


In [7]:
tovector_input = pd.DataFrame(data_grouped_by_year_label[["Label", "title_words"]].groupby(data_grouped_by_year_label.Label)["title_words"].apply(list))
tovector_input["words"] = tovector_input["title_words"].progress_apply(lambda x: " ".join(x))

Progress: 100%|██████████| 249/249 [00:04<00:00, 60.33it/s]


In [8]:
len(tovector_input)

249

In [9]:
tovector_input.head()

,title_words,words
Label,,
Acoustics,[psychological measurements annoyance related ...,psychological measurements annoyance related p...
Agricultural Economics & Policy,[methods maintaining fertility successful jers...,methods maintaining fertility successful jerse...
Agricultural Engineering,[studies high temperature drying discoloration...,studies high temperature drying discoloration ...
"Agriculture, Dairy & Animal Science",[propionic acid sodium propionate calcium prop...,propionic acid sodium propionate calcium propi...
"Agriculture, Multidisciplinary",[variation chemical composition mangels bearin...,variation chemical composition mangels bearing...


In [10]:
vectorizer = CountVectorizer()
vectorized_words = vectorizer.fit_transform(tovector_input.words)

In [12]:
count_vocab = {value:key for key,value in vectorizer.vocabulary_.items()}
count_rev_dict = {value:key for key,value in vectorizer.vocabulary_.items()}
words = [count_rev_dict[i] for i in range(len(vectorizer.vocabulary_.keys()))]

In [13]:
vectorized_wordarray = vectorized_words.toarray()
document_frequency = (vectorized_wordarray != 0).sum(0).tolist()
document_frequency_log =  [np.log(x) for x in document_frequency]
#2. Term Frequency - Number of times in which a word appears in a document
term_frequency = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
term_frequency_log = [np.log(x) for x in term_frequency]

In [14]:
vectorized_wordarray.shape

(249, 2034183)

In [15]:
data_grouped_by_year_label["number_of_pubs"] = data_grouped_by_year_label["title_without_stopwords"].progress_apply(lambda x: len(x))

Progress: 100%|██████████| 19980/19980 [00:00<00:00, 568023.38it/s]


In [16]:
data_grouped_by_year_label["word_count"] = data_grouped_by_year_label["title_words"].progress_apply(lambda x: len(x.split()))

Progress: 100%|██████████| 19980/19980 [00:49<00:00, 403.90it/s]


In [17]:
data_grouped_by_year_label.head()

,index,Label,PubYear,title_without_stopwords,cleaned_titles,title_words_,title_words,number_of_pubs,word_count
0,0,Acoustics,1929,"[hearing speech auditoriums, hearing speech au...",[psychological measurements annoyance related ...,psychological measurements annoyance related p...,psychological measurements annoyance related p...,16,40
1,1,Acoustics,1930,[reverberation time bridge instrumental metho...,"[theory horn type loud speaker, discussion sym...",theory horn type loud speaker discussion sympo...,theory horn type loud speaker discussion sympo...,112,250
2,2,Acoustics,1931,"[some physical characteristics speech music, s...","[microphone technique radio broadcasting, orga...",microphone technique radio broadcasting organ ...,microphone technique radio broadcasting organ ...,66,178
3,3,Acoustics,1932,"[position finding under water sound signals, p...",[acoustic pick up philadelphia orchestra broad...,acoustic pick up philadelphia orchestra broadc...,acoustic pick up philadelphia orchestra broadc...,68,159
4,4,Acoustics,1933,"[some notes character bell tones, some notes c...","[sound absorption non reactive mixtures, deter...",sound absorption non reactive mixtures determi...,sound absorption non reactive mixtures determi...,62,164


In [18]:
#word_ = "Dextromoramide"

#word_ = "purposed"
#word_ = "quayside"

word_ = "chiaroscuro"


index = words.index(word_)
count_rev_dict[index]
counts = vectorized_wordarray[:,index]

discipline_indices = [discipline_index for discipline_index in range(len(counts.tolist())) if counts[discipline_index] != 0]

counts_for_disciplines = {discipline_index: count for discipline_index, count in enumerate(counts.tolist())}
sorted_counts = sorted(counts_for_disciplines.items(), key=operator.itemgetter(1), reverse=True)

total_disciplines_present = len([count for i, count in enumerate(counts.tolist()) if count > 0])
to_show_count = 5

top_5_disciplines = [index for (index, count) in sorted_counts[:5]]
if total_disciplines_present < 5:
    top_5_disciplines = [index for (index, count) in sorted_counts[:total_disciplines_present]]
     
#disciplines = [list(tovector_input.index)[index] for index in discipline_indices]
disciplines = tovector_input.reset_index()["Label"]
present_dis = [disciplines[i] for i in top_5_disciplines]
print("Total disciplines present",len([count for i, count in enumerate(counts.tolist()) if count > 0]))
print("Present in:", present_dis)
print("Count:", len(disciplines))

Total disciplines present 22
Present in: ['Social Sciences, Interdisciplinary', 'Area Studies', 'Communication', 'Economics', 'Energy & Fuels']
Count: 249


In [19]:
def word_counts2(discipline, word):

    _data = data_grouped_by_year_label[data_grouped_by_year_label.Label == discipline]
    
    _data["selected_word_count"] = _data["title_words"].apply(lambda x: x.split().count(word))
    _data["normalized_selected_word_count"] = _data.apply(lambda x: float(x["selected_word_count"])/x["number_of_pubs"], axis=1) 
    
    return _data["PubYear"].tolist(), _data["normalized_selected_word_count"].tolist()

In [20]:
years = []
freqs = []

for dis in present_dis:
    
    _ys, _fqs = word_counts2(dis, word_)
    
    years.append(_ys)
    freqs.append(_fqs)

In [21]:
datatoplot = [go.Scatter(
            x=years[0],
            y=freqs[0],
            mode = 'lines+markers',
            name=present_dis[0],
    ),go.Scatter(
            x=years[1],
            y=freqs[1],
            mode='lines',
            name=present_dis[1],
    ),go.Scatter(
            x=years[2],
            y=freqs[2],
            mode='lines',
            name=present_dis[2],
    ),go.Scatter(
            x=years[3],
            y=freqs[3],
            mode='lines',
            name=present_dis[3],
    ),go.Scatter(
            x=years[4],
            y=freqs[4],
            mode='lines',
            name=present_dis[4]
    )]




layout = go.Layout(
    title='Word Propagation ' + word_,
    
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Popularity Ratio',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    )
)


fig1 = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig1, filename='freq')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

